In [1]:
import datetime
from datetime import date
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [42]:
def Finding(Way, path):
    newval = driver.find_element(By.Way, path)
    return newval

In [46]:
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
chrome_options = webdriver.ChromeOptions()

#chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)
time.sleep(2)

C:\Users\sungj\AppData\Local\Temp\ipykernel_9724\1025757986.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)
C:\Users\sungj\AppData\Local\Temp\ipykernel_9724\1025757986.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)


In [51]:
driver.implicitly_wait(10)

url = "https://dart.fss.or.kr/dsab007/main.do"
driver.get(url)
driver.implicitly_wait(10)
selectpath = Select(driver.find_element(By.XPATH, "//*[@id='option']"))
selectpath.select_by_index(2)
input = driver.find_element(By.XPATH, '//*[@id="reportName"]')
input.send_keys('신규시설투자등')
input.send_keys(Keys.RETURN)
driver.implicitly_wait(10)
time.sleep(2)
#검색도착
conamelist = []
hreflist = []
for k in range(10):        
    for i in range(1,16):
        
        tds = driver.find_element(By.XPATH, f'//*[@id="tbody"]/tr[{i}]/td[3]')
        a = tds.find_element(By.TAG_NAME, 'a')
        href = a.get_attribute('href')
        coname = driver.find_element(By.XPATH, '*[@id="tbody"]/tr[{i}]/td[4]').get_attribute('text')
        hreflist.append(href)
        conamelist.append(coname)
    driver.find_element(By.XPATH, f'//*[@id="psWrap"]/div[2]/ul/li[{k+4}]').send_keys(Keys.ENTER)
    


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression *[@id="tbody"]/tr[{i}]/td[4] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '*[@id="tbody"]/tr[{i}]/td[4]' is not a valid XPath expression.
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x006B6643]
	(No symbol) [0x0064BE21]
	(No symbol) [0x0054DA9D]
	(No symbol) [0x005509E4]
	(No symbol) [0x005508AD]
	(No symbol) [0x00550B30]
	(No symbol) [0x00580FAC]
	(No symbol) [0x0058147B]
	(No symbol) [0x005B8DC2]
	(No symbol) [0x0059FDC4]
	(No symbol) [0x005B6B09]
	(No symbol) [0x0059FB76]
	(No symbol) [0x005749C1]
	(No symbol) [0x00575E5D]
	GetHandleVerifier [0x0092A142+2497106]
	GetHandleVerifier [0x009585D3+2686691]
	GetHandleVerifier [0x0095BB9C+2700460]
	GetHandleVerifier [0x00763B10+635936]
	(No symbol) [0x00654A1F]
	(No symbol) [0x0065A418]
	(No symbol) [0x0065A505]
	(No symbol) [0x0066508B]
	BaseThrea

In [ ]:
driver.get(href)
driver.implicitly_wait(10)

In [49]:
coname

NameError: name 'coname' is not defined

In [31]:
coname = Finding(XPATH, )
invcost = 
mymoney
mymoneyrat

In [25]:
linklist = []
for link in hrefs:
    link = link.get_attribute('a')
    link = link.get_attribute('href')
    linklist.append(link)
linklist

AttributeError: 'NoneType' object has no attribute 'get_attribute'

In [ ]:
//*[@id="reportName"]